### Package Python

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os as os
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import autocorrelation_plot
from sklearn import linear_model

### Settings Jupyter

In [2]:
# set the max columns to none
pd.set_option('display.max_columns', None)

### Functions

In [3]:
def plotar(titulo: str, labelx: str, labely: str, x: str, y: str, dataset: dict, fontt: int, fontlx: int, fontly: int) -> dict: 
    """Função para plotagem de gráfico"""
    sns.set_palette('Accent')
    sns.set_style('darkgrid')
    ax = sns.lineplot(x = x, y = y, data = dataset)
    ax.figure.set_size_inches(12, 6)
    ax.set_title(titulo, loc='left', fontsize=fontt)
    ax.set_xlabel(labelx, fontsize=fontlx)
    ax.set_ylabel(labely, fontsize=fontly)
    ax = ax
    return ax

In [4]:
def plot_comparacao(x, y1,y2,y3, y4=None, params=False, dataset=None, titulo=None):
    plt.figure(figsize=(16,12))
    if params:
        ax = plt.subplot(4,1,1)
        ax.set_title(titulo, fontsize=18, loc='left')
        sns.lineplot(x = x, y = y1, data = dataset) 
        ax = plt.subplot(4,1,2)
        sns.lineplot(x = x, y = y2, data = dataset) 
        ax = plt.subplot(4,1,3)
        sns.lineplot(x = x, y = y3, data = dataset) 
        ax = plt.subplot(4,1,4)
        sns.lineplot(x = x, y = y4, data = dataset)     
    else:
        ax = plt.subplot(3,1,1)
        ax.set_title(titulo, fontsize=18, loc='left')
        sns.lineplot(x = x, y = y1, data = dataset) 
        ax = plt.subplot(3,1,2)
        sns.lineplot(x = x, y = y2, data = dataset) 
        ax = plt.subplot(3,1,3)
        sns.lineplot(x = x, y = y3, data = dataset)  
    ax = ax
    return ax

### Objective of this notebook.

This analyze aims to explore the ANAC database of flights from January 2022. It was used basic database for this study.

### Reading of bases

In [5]:
_path = os.getcwd()
_path

'c:\\git\\Gol_Cases'

In [6]:
_path1 = os.path.join(_path, 'data', 'basica2022-01.txt')
_path1

'c:\\git\\Gol_Cases\\data\\basica2022-01.txt'

In [7]:
_df = pd.read_csv(_path1, sep=';', encoding='latin1')
_df.shape

C:\Users\conta\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(69597, 110)

* Cleaning not necessary data for analyze.

In [8]:
_df = _df.drop(columns=[
'sg_empresa_icao',	
'sg_empresa_iata',
'nr_singular',	
'id_di',	
'cd_di',
'ds_di',
'ds_grupo_di', 
'nr_ano_referencia',
'nr_semestre_referencia',
'nr_trimestre_referencia',
'nr_ano_mes_referencia',
'id_tipo_linha',
'cd_tipo_linha',
'ds_tipo_linha',
'ds_natureza_tipo_linha',
'ds_servico_tipo_linha',
'nr_ano_partida_real',
'nr_semestre_partida_real',
'nm_semestre_partida_real',
'nr_trimestre_partida_real',
'nm_trimestre_partida_real',
'nr_mes_partida_real',
'nm_mes_partida_real',
'nr_semana_partida_real',
'nm_dia_semana_partida_real',
'nr_ano_mes_partida_real',
'sg_icao_origem',
'sg_uf_origem',
'nm_regiao_origem',	
'nr_ano_chegada_real',	
'nr_semestre_chegada_real',
'nm_semestre_chegada_real',
'nr_trimestre_chegada_real',
'nm_trimestre_chegada_real',
'nr_mes_chegada_real',
'nm_mes_chegada_real',
'nr_ano_mes_chegada_real',
'ds_matricula',
'sg_uf_destino',
'nm_regiao_destino',
'nr_escala_destino',
'lt_combustivel',
'kg_bagagem_livre',
'kg_bagagem_excesso',
'nr_decolagem',
'id_arquivo',
'nm_arquivo',
'nr_linha',	
'dt_sistema'])
_df.shape

(69597, 61)

In [9]:
_df = _df.rename(columns={'nr_assentos_ofertados':'pax_ofertados', 'nr_passag_pagos':'pax', 'nr_rpk':'rpk'})

In [10]:
_df.dtypes

id_basica                int64
id_empresa               int64
nm_empresa              object
nm_pais                 object
ds_tipo_empresa         object
                         ...  
nr_bagagem_gratis_km     int64
nr_ask                   int64
rpk                      int64
nr_atk                   int64
nr_rtk                   int64
Length: 61, dtype: object

Verifying nan values in the columns.

In [11]:
_df.describe()

,id_basica,id_empresa,nr_voo,nr_mes_referencia,nr_semana_referencia,nr_dia_referencia,nr_dia_partida_real,id_aerodromo_origem,nr_etapa,nr_semana_chegada_real,nr_dia_chegada_real,id_equipamento,id_aerodromo_destino,pax_ofertados,kg_payload,km_distancia,pax,nr_passag_gratis,kg_carga_paga,kg_carga_gratis,kg_correio,kg_peso,nr_pax_gratis_km,nr_carga_paga_km,nr_carga_gratis_km,nr_correio_km,nr_bagagem_paga_km,nr_bagagem_gratis_km,nr_ask,rpk,nr_atk,nr_rtk
count,6.959700e+04,6.959700e+04,69597.000000,69597.0,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69596.000000,69596.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,69597.000000,6.959700e+04,6.959700e+04,6.959700e+04,6.959700e+04,6.959700e+04,6.959700e+04,6.959700e+04,6.959700e+04,69597.000000
mean,2.264191e+07,1.000946e+06,3490.833944,1.0,3.571246,15.898473,15.901504,621.361898,1.055591,3.588022,15.910225,187.901835,623.448583,154.157967,19613.930442,1494.948403,123.306163,1.892467,1667.116973,6.296378,97.127965,12290.853686,3125.178312,8.564423e+06,1.710049e+04,1.611728e+05,1.201916e+04,1.530424e+06,2.819528e+05,2.241117e+05,4.614829e+04,28228.931750
std,2.053645e+04,3.945483e+02,2255.404553,0.0,1.300965,8.905715,8.904413,1884.908792,0.292194,1.298763,8.898590,188.504497,1892.198097,66.304140,13506.770209,1795.244317,57.833278,2.585987,7688.578916,71.073279,963.443954,8684.315281,10002.984847,4.940851e+07,3.720656e+05,1.777468e+06,3.733798e+04,4.861559e+06,5.496425e+05,4.221822e+05,1.124892e+05,70439.806904
min,2.260455e+07,1.000002e+06,2.000000,1.0,1.000000,1.000000,1.000000,5.000000,0.000000,1.000000,1.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,2.262400e+07,1.000854e+06,1802.000000,1.0,2.000000,8.000000,8.000000,218.000000,1.000000,2.000000,8.000000,16.000000,218.000000,118.000000,14998.000000,496.000000,95.000000,0.000000,0.000000,0.000000,0.000000,8654.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.065840e+05,5.931200e+04,4.501800e+04,7.160000e+03,4264.000000
50%,2.264201e+07,1.000909e+06,3178.000000,1.0,4.000000,16.000000,16.000000,427.000000,1.000000,4.000000,16.000000,101.000000,427.000000,170.000000,18165.000000,874.000000,130.000000,1.000000,53.000000,0.000000,0.000000,11432.000000,757.000000,3.782300e+04,0.000000e+00,0.000000e+00,0.000000e+00,6.377040e+05,1.442100e+05,1.085850e+05,1.628300e+04,9865.000000
75%,2.265981e+07,1.001093e+06,4432.000000,1.0,5.000000,24.000000,24.000000,633.000000,1.000000,5.000000,24.000000,272.000000,633.000000,186.000000,19800.000000,1800.000000,160.000000,3.000000,445.000000,0.000000,0.000000,14188.000000,2495.000000,4.392360e+05,0.000000e+00,0.000000e+00,7.686000e+03,1.630210e+06,2.977920e+05,2.490180e+05,3.473500e+04,23021.000000
max,2.267771e+07,1.007484e+06,9998.000000,1.0,6.000000,31.000000,31.000000,21967.000000,7.000000,6.000000,31.000000,578.000000,21967.000000,516.000000,132631.000000,12217.000000,423.000000,39.000000,112046.000000,7790.000000,29061.000000,112046.000000,264908.000000,9.809300e+08,5.943020e+07,1.735520e+08,1.328650e+06,9.677660e+07,6.303970e+06,5.167790e+06,1.230060e+06,980930.000000


In [12]:
_counts_cols = _df.count().reset_index()
_counts_cols.columns = ['columns', 'values']
_counts_cols

,columns,values
0,id_basica,69597
1,id_empresa,69597
2,nm_empresa,69597
3,nm_pais,69597
4,ds_tipo_empresa,69597
...,...,...
56,nr_bagagem_gratis_km,69597
57,nr_ask,69597
58,rpk,69597
59,nr_atk,69597


In [13]:
_counts_cols[_counts_cols['values']!=69597]

,columns,values
19,sg_iata_origem,69588
25,hr_chegada_real,69595
27,nr_semana_chegada_real,69596
28,nm_dia_semana_chegada_real,69596
29,nr_dia_chegada_real,69596
35,sg_iata_destino,69588
48,nr_horas_voadas,69595
50,nr_velocidade_media,69595


In [14]:
_df =_df.fillna(0)

Removing white spaces.

In [15]:
_cols = _df.select_dtypes(include='object')
_cols = _cols.columns.to_list()
for i in _cols:
    _df[i] = _df[i].str.strip()


In [16]:
_df.loc[19].to_list()

[22675653,
 1000358,
 'AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA',
 'COLÔMBIA',
 'ESTRANGEIRA REGULAR',
 86,
 '2022-01-11',
 '1º SEMESTRE',
 '1º TRIMESTRE',
 1,
 'JANEIRO',
 3,
 'TERÇA-FEIRA',
 11,
 'INTERNACIONAL',
 '07:28:00',
 '2022-01-11',
 11,
 301,
 'GRU',
 'GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO',
 'GUARULHOS',
 'BRASIL',
 'AMÉRICA DO SUL',
 1,
 '13:19:00',
 '2022-01-11',
 3.0,
 'TERÇA-FEIRA',
 11.0,
 15,
 'A320',
 'AIRBUS A320-100/200',
 111,
 'SKBO',
 'BOG',
 'EL DORADO INTERNATIONAL AIRPORT',
 'BOGOTÁ',
 'COLÔMBIA',
 'AMÉRICA DO SUL',
 150,
 17600,
 4336,
 150,
 0,
 57,
 0,
 0,
 '5,85',
 13557,
 '741,17',
 0,
 247152,
 0,
 0,
 0,
 0,
 650400,
 650400,
 76313,
 58783]